In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
np.set_printoptions(suppress=True)
sns.set(style='whitegrid')

In [2]:
train_path = 'data/train.csv'
test_path = 'data/test.csv'
train_data = pd.read_csv(train_path, index_col='Id')
test_data = pd.read_csv(test_path, index_col='Id')
features = train_data.iloc[:,:-1]
target = train_data.loc[:, ['SalePrice']]
print('Train Set Size : ', train_data.shape)
print('Test Set Size : ', test_data.shape)
print('Train Features Size : ', features.shape)
num_train_rows = train_data.shape[0]
num_test_rows = test_data.shape[0]
print('Train Rows : ', num_train_rows)
print('Test Rows : ', num_test_rows)

all_data = pd.concat((features, test_data)).reset_index(drop=True)

Train Set Size :  (1460, 80)
Test Set Size :  (1459, 79)
Train Features Size :  (1460, 79)
Train Rows :  1460
Test Rows :  1459


In [3]:
# Analyze SalePrice
print('Skewness of SalePrice before Log Transform : %f'% target.skew())
print('Kurtosis of SalePrice before Log Transform : %f'% target.kurt())

'''
Skew = 1.882876 indicates positive skew with tail to the right.
Kurt = 6.536282 indicates heavy tails i.e. more data on tails.
'''

#Apply Log transformation
target['SalePrice'] = np.log(target['SalePrice'])
print('Skewness of SalePrice after Log Transform : %f'% target.skew())
print('Kurtosis of SalePrice after Log Transform : %f'% target.kurt())

Skewness of SalePrice before Log Transform : 1.882876
Kurtosis of SalePrice before Log Transform : 6.536282
Skewness of SalePrice after Log Transform : 0.121335
Kurtosis of SalePrice after Log Transform : 0.809532


In [4]:
# Missing Data
null_features = all_data.columns[all_data.isnull().any()]
missing_ratio = (all_data[null_features].isnull().sum()/len(all_data)) * 100
missing_data = pd.DataFrame({'Missing Ratio' :missing_ratio})
print(missing_data.sort_values(by='Missing Ratio',ascending=False))


for col in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu']:
    all_data[col] = all_data[col].fillna('None')

for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'MasVnrType']:
    all_data[col] = all_data[col].fillna('None')
    
for col in ['BsmtExposure', 'BsmtFinType2', 'BsmtQual', 'BsmtCond', 'BsmtFinType1']:
    all_data[col] = all_data[col].fillna('None')

'''
No GarageYrBlt means no Garage. We can impute mean/median since it would 
incorrectly convey existence of Garage. same reasoning for MasVnrArea.
'''
for col in ['GarageYrBlt', 'MasVnrArea']:
    all_data[col] = all_data[col].fillna(0)

'''
Group data by neighborhood & imputed null LotFrontage columns with median of
grouped data.
'''
all_data['LotFrontage'] = all_data.groupby(['Neighborhood'])\
                    ['LotFrontage'].transform(lambda x : x.fillna(x.median()))
    
all_data['Electrical'] = \
    all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
    
all_data['MSZoning'] = \
    all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
    
all_data['Utilities'] = all_data['Utilities'].fillna('ELO')

all_data['Exterior1st'] = all_data['Exterior1st'].fillna('Other')

all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna('Other')

all_data['SaleType'] = all_data['SaleType'].fillna('Oth')

all_data['Functional'] = \
    all_data['Functional'].fillna(all_data['Functional'].mode()[0])

all_data['KitchenQual'] = \
    all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

for col in ['BsmtFullBath', 'BsmtHalfBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea']:
    all_data[col] = all_data[col].fillna(0)
    
null_features = all_data.columns[all_data.isnull().any()]
missing_ratio = (all_data[null_features].isnull().sum()/len(all_data)) * 100
missing_data = pd.DataFrame({'Missing Ratio' :missing_ratio})
print(missing_data.sort_values(by='Missing Ratio',ascending=False))

              Missing Ratio
PoolQC                99.66
MiscFeature           96.40
Alley                 93.22
Fence                 80.44
FireplaceQu           48.65
LotFrontage           16.65
GarageFinish           5.45
GarageQual             5.45
GarageCond             5.45
GarageYrBlt            5.45
GarageType             5.38
BsmtExposure           2.81
BsmtCond               2.81
BsmtQual               2.77
BsmtFinType2           2.74
BsmtFinType1           2.71
MasVnrType             0.82
MasVnrArea             0.79
MSZoning               0.14
BsmtFullBath           0.07
BsmtHalfBath           0.07
Functional             0.07
Utilities              0.07
GarageArea             0.03
GarageCars             0.03
Electrical             0.03
KitchenQual            0.03
TotalBsmtSF            0.03
BsmtUnfSF              0.03
BsmtFinSF2             0.03
BsmtFinSF1             0.03
Exterior2nd            0.03
Exterior1st            0.03
SaleType               0.03
Empty DataFrame
Colu

In [5]:
num_features = all_data.select_dtypes(include=np.number)
num_features_std = (num_features - num_features.mean())/num_features.std()
train_data_num_features = num_features_std[:num_train_rows]
test_data_num_features = num_features_std[num_train_rows:]

print(train_data_num_features.columns)

from sklearn.feature_selection import RFE
from sklearn.linear_model import Ridge
regressor = Ridge(alpha=0.05, normalize=True)
rfe = RFE(regressor, 11)
rfe = rfe.fit(train_data_num_features, target)
print(rfe.support_)
print(rfe.ranking_)


Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')
[False False False  True  True  True  True False False False False  True
  True  True False  True  True False False False False False False  True
 False  True False False False False False False False False False False]
[ 2 25  9  1  1  1  1 20 11 19 18  1  1  1 21  1  1 17  4  8 26  7  3  1
 15  1 10  6 24 14 16  5 12 22 23 13]


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [6]:
'''
RFE Selected features without Standardization
OverallQual, OverallCond, BsmtFullBath, BsmtHalfBath, FullBath, HalfBath, 
KitchenAbvGr, TotRmsAbvGrd, Fireplaces, GarageCars, YrSold
'''
final_num_features = all_data.loc[:,['OverallQual', 'OverallCond', 'BsmtFullBath', 'BsmtHalfBath',
                              'FullBath', 'HalfBath', 'KitchenAbvGr',
                              'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'YrSold']]

train_data_num_features = final_num_features[:num_train_rows]
test_data_num_features = final_num_features[num_train_rows:]

from sklearn.cross_validation import train_test_split
X_train , X_test, y_train, y_test = train_test_split(train_data_num_features, target, test_size=0.4, random_state=0)

from sklearn.linear_model import Ridge
from sklearn import metrics
regressor = Ridge()
regressor.fit(X_train, y_train)

y_train_predict = regressor.predict(X_test)
print('MAE',metrics.mean_absolute_error(y_test, y_train_predict))
print('MSE',metrics.mean_squared_error(y_test, y_train_predict))
print('RMSE',np.sqrt(metrics.mean_squared_error(y_test, y_train_predict)))
print('RMSLE',np.sqrt(metrics.mean_squared_log_error(y_test, y_train_predict)))

print("Accuracy --> ", regressor.score(X_test, y_test)*100)

MAE 0.12248659571111567
MSE 0.027138107686690522
RMSE 0.1647364795262134
RMSLE 0.01261393849415613
Accuracy -->  82.030072018874


c:\users\revan\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
'''
RFE Selected features WITH Standardization
OverallQual, OverallCond, YearBuilt, YearRemodAdd, TotalBsmtSF, 1stFlrSF, 
2ndFlrSF, GrLivArea, BsmtFullBath, GarageCars, Fireplaces
'''
final_std_num_features = num_features_std.loc[:,['OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
                              'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                              'GrLivArea', 'Fireplaces', 'GarageCars', 'BsmtFullBath']]

train_data_std_num_features = final_std_num_features[:num_train_rows]
test_data_std_num_features = final_std_num_features[num_train_rows:]

from sklearn.cross_validation import train_test_split
X_train , X_test, y_train, y_test = train_test_split(train_data_std_num_features, target, test_size=0.4, random_state=0)

from sklearn.linear_model import Ridge
from sklearn import metrics
regressor = Ridge()
regressor.fit(X_train, y_train)

y_train_predict = regressor.predict(X_test)
print('MAE',metrics.mean_absolute_error(y_test, y_train_predict))
print('MSE',metrics.mean_squared_error(y_test, y_train_predict))
print('RMSE',np.sqrt(metrics.mean_squared_error(y_test, y_train_predict)))
print('RMSLE',np.sqrt(metrics.mean_squared_log_error(y_test, y_train_predict)))

print("Accuracy --> ", regressor.score(X_test, y_test)*100)

MAE 0.09586879909480285
MSE 0.02676859763736878
RMSE 0.16361111709590145
RMSLE 0.012168109652123912
Accuracy -->  82.27474895255243
